In [1]:
import xml.etree.ElementTree as ET
import numpy as np
from datetime import datetime
import pandas as pd

tree = ET.parse('pubmed_sample.xml')
root = tree.getroot()

In [2]:
def journalDictHelper(article):
    journal = article.find('./Journal')
    journalDict = {}
    for elem in list(journal.iter()):
        if elem.text.isspace():
            journalDict[elem.tag] = elem.attrib
        else:
            journalDict[elem.tag] = elem.text
    
    return journalDict

In [3]:
def historyDictHelper(history):
    
    historyDict = {}
    historyNode = list(history)
    for pubDateNode in historyNode:
        pubDateDict = {}
        for elem in list(pubDateNode.iter()):
            if elem.text.isspace():
                pubDateDict.update(elem.attrib)
            else:
                pubDateDict[elem.tag] = elem.text
                
        if("Year" in pubDateDict and "Month" in pubDateDict and "Day" in pubDateDict):
            datestr = pubDateDict["Year"] + '-' + pubDateDict["Month"] + '-' + pubDateDict["Day"]
            pubDateDict["Date"] = datetime.strptime(datestr, '%Y-%m-%d').date().strftime('%m-%d-%Y')
    
        historyDict.update({pubDateDict['PubStatus']: pubDateDict['Date']})
        
    return historyDict

In [4]:
def authorDictHelper(article):
    
    authorDict = {}
    i = 0
    author = article.find('./AuthorList')
    authorListNode = list(author)
    for authorNode in authorListNode:
        currentAuthor = {}
        for elem in list(authorNode.iter()):
            currentAuthor[elem.tag] = elem.text
        currentAuthor.pop('Author', None)
        currentAuthor.pop('Affiliation', None)
        currentAuthor.pop('Initials', None)
        authorDict.update({i:currentAuthor})
        i+=1
        
    return authorDict

In [5]:
def articleDictHelper(article):
    
    articleDict = {}
    
    if(article.find('./Journal/ISSN') != None):
        issn = article.find('./Journal/ISSN').text
        articleDict['ISSN'] = issn
        
    if(article.find('./ArticleTitle') != None):
        articleTitle = article.find('./ArticleTitle').text
        articleDict['Title'] = articleTitle
        
    if(article.find('./ArticleDate') != None):
        articleDateNode = article.find('./ArticleDate')
    
    for child in articleDateNode:
        articleDict[child.tag] = child.text
        
    if("Year" in articleDict and "Month" in articleDict and "Day" in articleDict):
        datestr = articleDict["Year"] + '-' + articleDict["Month"] + '-' + articleDict["Day"]
        articleDict["Date"] = datetime.strptime(datestr, '%Y-%m-%d').date().strftime('%m-%d-%Y')
        articleDict.pop('Year')
        articleDict.pop('Month')
        articleDict.pop('Day')
    
    return articleDict

In [30]:
import sqlite3
conn = sqlite3.connect('P3.db')
print("successful connection")

conn.execute("DROP TABLE IF EXISTS author;")
conn.execute("DROP TABLE IF EXISTS history;")
conn.execute("DROP TABLE IF EXISTS article;")
conn.execute("DROP TABLE IF EXISTS authorArticle;")
conn.execute("DROP TABLE IF EXISTS journal;")
conn.execute("DROP TABLE IF EXISTS pubStatus;")

conn.execute('''CREATE TABLE Author (
  authorID INTEGER PRIMARY KEY AUTOINCREMENT,
  firstName VARCHAR(40),
  lastName VARCHAR(40),
  UNIQUE(firstName, lastName)
);''')

conn.execute('''CREATE TABLE Journal (
  journalID INTEGER PRIMARY KEY AUTOINCREMENT,
  issn VARCHAR(25),
  volume INTEGER,
  issue INTEGER,
  pubYear DATE VARCHAR(40),
  pubMonth MONTH VARCHAR(40),
  title LONGTEXT
);''')

conn.execute('''Create TABLE Article (
  articleID INTEGER PRIMARY KEY AUTOINCREMENT,
  journalID INTEGER,
  title LONGTEXT,
  date DATE,
  FOREIGN KEY (journalID) REFERENCES journal(journalID)
);''')

conn.execute('''CREATE TABLE AuthorArticle (
  authorArticleID INTEGER PRIMARY KEY AUTOINCREMENT,
  authorID INTEGER,
  articleID INTEGER,
  FOREIGN KEY (authorID) REFERENCES author(authorID)
  FOREIGN KEY (articleID) REFERENCES article(articleID)
);''')

conn.execute('''CREATE TABLE PubStatus (
  pubStatusID INTEGER PRIMARY KEY AUTOINCREMENT,
  statusName VARCHAR(40) UNIQUE
);''')

conn.execute('''CREATE tABLE History (
  historyID INTEGER PRIMARY KEY AUTOINCREMENT,
  articleID INTEGER,
  pubStatusID INTEGER,
  date DATE,
  FOREIGN KEY (articleID) REFERENCES article(articleID)
  FOREIGN KEY (pubStatusID) REFERENCES pubStatus(pubStatusID)
);''')

conn.close()
print("Tables created successfully")

successful connection
Tables created successfully


In [12]:
def journalInsertion(journalDict):
    
    conn = sqlite3.connect('P3.db')
    issn = journalDict['ISSN']
    volume = journalDict['Volume']
    issue = journalDict['Issue']
    year = journalDict['Year']
    month = journalDict['Month']
    title = journalDict['Title']
    
    conn.execute("INSERT INTO " +
                 "journal(issn, volume, issue, pubYear, pubMonth, title) " +
                 "VALUES(?,?,?,?,?,?)", 
                 (issn, volume, issue, year, month, title))
    conn.commit()
    
    cur = conn.cursor()
    cur.execute("SELECT * FROM journal")
    print(cur.fetchall())
    conn.close()

In [17]:
def articleInsertion(articleDict):
    
    conn = sqlite3.connect('P3.db')
    issn = articleDict['ISSN']
    title = articleDict['Title']
    date = articleDict['Date']
        
    cur = conn.cursor()
    cur.execute("SELECT journalID FROM journal WHERE issn = ?", (issn,))
    
    journalID = cur.fetchone()[0]
    
    conn.execute("INSERT INTO article(journalID, title, date) VALUES (?,?,?)",
                (journalID, title, date))
    conn.commit()
    
    cur.execute("SELECT articleID FROM article WHERE journalID = ? AND title = ? AND date = ?",
               (journalID, title, date))
    articleID = cur.fetchall()[0][0]
    conn.close()
    return articleID

In [45]:
def authorInsertion(authorDict, articleID):
    
    conn = sqlite3.connect("P3.db")
    for number, author in authorDict.items():
        firstname = author['ForeName']
        lastname = author['LastName']
        
        conn.execute("INSERT OR IGNORE INTO author(firstname, lastname) VALUES(?,?)",
                     (firstname, lastname))
        conn.commit()
        
        cur = conn.cursor()
        cur.execute("SELECT authorID FROM author WHERE firstname = ? AND lastname = ?", (firstname, lastname))
        authorID = cur.fetchone()[0]
        
        conn.execute("INSERT OR IGNORE INTO authorArticle(authorID, articleID) VALUES(?,?)",
                     (authorID, articleID))
        conn.commit()
        
    cur.execute("SELECT * FROM author")
    print(cur.fetchall())
    
    cur.execute("SELECT * FROM authorArticle")
    print(cur.fetchall())
    
    conn.close()

In [46]:
def historyInsertion(historyDict, articleID):
    
    conn = sqlite3.connect("P3.db")
    for pubStatus, date in historyDict.items():
        conn.execute("INSERT OR IGNORE INTO pubStatus(statusName) VALUES(?)", (pubStatus,))
        conn.commit()
        
        cur = conn.cursor()
        cur.execute("SELECT pubStatusID FROM pubStatus WHERE statusName = ?", (pubStatus,))
        pubStatusID = cur.fetchone()[0]
        
        conn.execute("INSERT INTO history(articleID, pubStatusID, date) VALUES (?,?,?)", (articleID, pubStatusID, date))
        conn.commit()
        
    cur.execute("SELECT * FROM pubStatus")
    print(cur.fetchall())
    
    cur.execute("SELECT * FROM history")
    print(cur.fetchall())        
    conn.close()

In [47]:
for base in root.findall("./"):

    if(base.find('.//Article') != None):
        article = base.find('.//Article')

        if(article.find('./Journal') != None):
            journalDict = journalDictHelper(article)
            journalInsertion(journalDict)

            # This is a check to see that it got everything correctly
            """
            for k,v in journalDict.items():
                print(k,v)
            """
            
            # TODO: Need to figure out what happens if there's no journal
            articleDict = articleDictHelper(article)
            articleID = articleInsertion(articleDict)
    
            # This is a check to see that it got everything correctly
            """
            for k,v in articleDict.items():
                print(k,v)
            """

        if(article.find('./AuthorList') != None):
            authorDict = authorDictHelper(article)
            authorInsertion(authorDict, articleID)
           
            """
            # This is a check to see that it got everything correctly
            for k,v in authorDict.items():
                for key, value in v.items():
                    print(k, key, value)
            """       

        
    if(base.find(".//History") != None):
        history = base.find('.//History')
        historyDict = historyDictHelper(history)
        historyInsertion(historyDict, articleID)

        # This is a check to see that it got everything correctly
        """
        for k,v in historyDict.items():
            print(k,v)
        """
    break

[(1, '1556-3316', 8, 2, '2012', 'Jul', 'HSS journal : the musculoskeletal journal of Hospital for Special Surgery'), (2, '1556-3316', 8, 2, '2012', 'Jul', 'HSS journal : the musculoskeletal journal of Hospital for Special Surgery'), (3, '1556-3316', 8, 2, '2012', 'Jul', 'HSS journal : the musculoskeletal journal of Hospital for Special Surgery'), (4, '1556-3316', 8, 2, '2012', 'Jul', 'HSS journal : the musculoskeletal journal of Hospital for Special Surgery'), (5, '1556-3316', 8, 2, '2012', 'Jul', 'HSS journal : the musculoskeletal journal of Hospital for Special Surgery'), (6, '1556-3316', 8, 2, '2012', 'Jul', 'HSS journal : the musculoskeletal journal of Hospital for Special Surgery'), (7, '1556-3316', 8, 2, '2012', 'Jul', 'HSS journal : the musculoskeletal journal of Hospital for Special Surgery'), (8, '1556-3316', 8, 2, '2012', 'Jul', 'HSS journal : the musculoskeletal journal of Hospital for Special Surgery')]
[(1, 'Cassie', 'Kuo'), (2, 'Alison', 'Edwards'), (3, 'Madhu', 'Mazumdar